# Loading SWESSAR Data

In [ ]:
# Import several libraries. 
# comments to the right could be useful for local installation on Windows.

from shapely import speedups      # https://www.lfd.uci.edu/~gohlke/pythonlibs/
speedups.disable()                # <-- handle a potential error in cartopy

import requests                
# !conda install -c anaconda requests 

# raster manipulation libraries
import rasterio                   # https://www.lfd.uci.edu/~gohlke/pythonlibs/
from osgeo import gdal            # https://www.lfd.uci.edu/~gohlke/pythonlibs/
import cartopy.crs as ccrs        # https://www.lfd.uci.edu/~gohlke/pythonlibs/
import rioxarray as rxr           # !conda install -c conda-forge rioxarray
import xarray as xr               # !conda install -c conda-forge xarray dask netCDF4 bottleneck

# plotting tools
from matplotlib import pyplot     # !conda install matplotlib
import datashader as ds           # https://www.lfd.uci.edu/~gohlke/pythonlibs/
import hvplot.xarray              # !conda install hvplot

# append the subfolders of the current working directory to pythons path

import os
import sys
import glob

swesarr_subdirs = ["data", "util"]
tmp = [sys.path.append(os.getcwd() + "/" + sd) for sd in swesarr_subdirs]
del tmp # suppress Jupyter notebook output, delete variable

from helper import gdal_corners, join_files, join_sar_radiom

In [ ]:
# select files to download

# SWESARR data website
source_repo = 'https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease/'

#Flight line and date of interest
flight_line = '/GRMCT2_31801_20007_016_200211_225_XX_01/'

In [ ]:
! wget -r -np -nH --reject "indexd.html*" -e robots=off -r --no-parent -A "*tif" source_repo + flight_line

In [ ]:
data_files = []
os.chdir(os.getcwd() + '/files/radar/SWESARR/prerelease/' + flight_line)
for file in glob.glob("*.tif"):
    data_files.append(file)

In [ ]:
data_files = [data for data in data_files if data[-8:] != '_dem.tif']

In [ ]:
data_files

**User to specify specific data to be pull**

In [ ]:
# store the location of the SAR tiles as they're located on the SWESARR data server
remote_tiles = [source_repo + flight_line + d for d in data_files]

# create local output data directory
# output_dir = os.getcwd() + '/files/radar/SWESARR/prerelease/' + flight_line
output_dir = os.getcwd() + '/'
try:
    os.makedirs(output_dir)
except FileExistsError:
    print('output directory prepared!')

# store individual TIF files locally on our computer / server
output_paths = [output_dir + d for d in data_files]

In [ ]:
##    for each file selected, store the data locally 
##
##    only run this block if you want to store data on the current 
##    server/hard drive this notebook is located.
##
################################################################

for remote_tile, output_path in zip(remote_tiles, output_paths):
    
    # download data
    r = requests.get(remote_tile)

    # Store data (~= 65 MB/file)
    if r.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(r.content)

In [ ]:
output_paths

In [ ]:
da = join_files(output_paths)
da

In [ ]:
# Set clim directly:
clim=(-20,20)
cmap='gray'
crs = ccrs.UTM(zone='12') #12n
tiles='OSM'
tiles='EsriImagery'

da.hvplot.image(x='x',y='y',groupby='band',cmap=cmap,clim=clim,rasterize=True,
                       xlabel='Longitude',ylabel='Latitude',
                       frame_height=500, frame_width=500,
                       xformatter='%.1f',yformatter='%.1f', crs=crs, tiles=tiles, alpha=0.8)